# CO2e WML Deployment Recipe

This notebook demonstrates the use of AI Model Factory recipes to deploy energy loss pipeline to WML

## Import libraries

In [1]:
import json
import yaml
import requests

## Set Variables

model_uri - URI to fetch the model from \
deployment_name - name to set for the deployment

In [2]:
model_factory_config_file_name = "../config/model_info.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

{'model_uri': 'runs:/0bf6e18c4069428bb395bb3e8eac2e54/energy_loss_to_CO2_epa', 's3_uri': 's3://testdataupload/9/d2dc1fcb1870406bb7f2d99e37fa823e/artifacts/energy_loss_to_CO2_epa', 'train_job_id': '43d1690c-a370-4ab3-8e97-8ae9fc4acbd5'}


In [4]:
model_uri = model_factory_config['s3_uri']
deployment_name = "CO2_model_112123_joe"

In [5]:
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"

In [6]:
url = endpoint_url + "/deployment/wml/create"

## Create a request to call WML deployment recipe

In [7]:
payload = {
    'endpoint': 'electrical_transformer_deployments',
    'deployment_name': deployment_name,
    'model_uri': model_uri,
    'flavor': 'sklearn',
    'software_spec_name': 'modelfactory'
}

In [8]:
files={
      "wml_credentials_file": ('wml-credentials.json',open('../../config/credentials/wml-credentials.json','rb'),'application/json')
}

In [9]:
headers = {
  'accept': 'application/json'
}

In [10]:
response = requests.request("POST", url, headers=headers, data=payload, files=files)

In [11]:
print(response.text)

{"job_id":"1cc125d6-5318-4917-8e73-4693876dcf4d","message":"WML deployment CO2_model_112123_joe submitted for creation"}


## Check the status of the job

In [12]:
job_id = json.loads(response.text)["job_id"]

In [13]:
url = endpoint_url + "/log" + f"/{job_id}"

In [14]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [15]:
print(json.loads(response.text)["logs"])

recipe_file_path /tmp/ray/session_2023-11-21_19-29-48_683486_1/runtime_resources/working_dir_files/_ray_pkg_2463abd887bfe395/Recipe.create
entrypoint_path /tmp/ray/session_2023-11-21_19-29-48_683486_1/runtime_resources/working_dir_files/_ray_pkg_2463abd887bfe395/main.py
2023-11-21 19:30:19,427	INFO worker.py:1334 -- Using address 172.63.0.4:6379 set in the environment variable RAY_ADDRESS
2023-11-21 19:30:19,427	INFO worker.py:1452 -- Connecting to existing Ray cluster at address: 172.63.0.4:6379...
2023-11-21 19:30:19,453	INFO worker.py:1627 -- Connected to Ray cluster. View the dashboard at 172.63.0.4:8265 
(run_recipe_on_ray pid=83, ip=172.63.0.5) 2023/11/21 19:30:20 INFO mlflow.tracking.fluent: Experiment with name '1cc125d6-5318-4917-8e73-4693876dcf4d' does not exist. Creating a new experiment.
(run_recipe_on_ray pid=83, ip=172.63.0.5) mlflow.experiment.name: 1cc125d6-5318-4917-8e73-4693876dcf4d
(run_recipe_on_ray pid=83, ip=172.63.0.5) mlflow.experiment.id: 25
(run_recipe_on_ray 

## Get the summary of the job

In [16]:
url = endpoint_url + "/summary" + f"/{job_id}"

In [17]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [18]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            if "status" in summary:
                html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code

In [29]:
print_job_details(job_id, endpoint_url)

### Check the summary

In [30]:
summary_url = endpoint_url + "/summary/"
get_response = requests.get(summary_url+job_id, headers={})
summary_data = get_response.json()
print(summary_data)

{'job_id': '1cc125d6-5318-4917-8e73-4693876dcf4d', 'status': 'DONE', 'summary': {'deployment_details': {'entity': {'asset': {'id': 'a79fb3c0-ea9a-407f-a866-f0ac9a6ddb2c', 'rev': '1'}, 'custom': {'AWS_ACCESS_KEY_ID': 'e6d28d7de535464da066c608726b4ee3', 'AWS_SECRET_ACCESS_KEY': '2fc6029cc5d799b6c572c0e730d6bc912c047cb1dddce409', 'MLFLOW_S3_ENDPOINT_URL': 'https://s3.us-south.cloud-object-storage.appdomain.cloud', 'MLFLOW_TRACKING_URI': 'http://mlflow:5000'}, 'deployed_asset_type': 'model', 'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580', 'name': 'XXS', 'num_nodes': 1}, 'name': 'CO2_model_112123_joe', 'online': {}, 'space_id': 'a0521ce5-5547-41e2-8495-66012d7e8780', 'status': {'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/339692d8-b809-4e30-b154-c3613ecf9d60/predictions'}, 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.cont

In [31]:
summary_data

{'job_id': '1cc125d6-5318-4917-8e73-4693876dcf4d',
 'status': 'DONE',
 'summary': {'deployment_details': {'entity': {'asset': {'id': 'a79fb3c0-ea9a-407f-a866-f0ac9a6ddb2c',
     'rev': '1'},
    'custom': {'AWS_ACCESS_KEY_ID': 'e6d28d7de535464da066c608726b4ee3',
     'AWS_SECRET_ACCESS_KEY': '2fc6029cc5d799b6c572c0e730d6bc912c047cb1dddce409',
     'MLFLOW_S3_ENDPOINT_URL': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
     'MLFLOW_TRACKING_URI': 'http://mlflow:5000'},
    'deployed_asset_type': 'model',
    'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580',
     'name': 'XXS',
     'num_nodes': 1},
    'name': 'CO2_model_112123_joe',
    'online': {},
    'space_id': 'a0521ce5-5547-41e2-8495-66012d7e8780',
    'status': {'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/339692d8-b809-4e30-b154-c3613ecf9d60/predictions'},
     'serving_urls': ['https://cpd-ibm-cpd

In [32]:
deployment_endpoint = summary_data['summary']['deployment_details']['entity']['status']

In [33]:
from datetime import datetime

deployment_endpoint['cp4d_url'] = deployment_endpoint['serving_urls'][0]
# Get the current date
current_date = datetime.now()

# Format the date and create the string
version_string = f"?version={current_date.strftime('%Y-%m-%d')}"
deployment_endpoint['cp4d_url'] = deployment_endpoint['cp4d_url'] + version_string

In [34]:
deployment_endpoint

{'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/339692d8-b809-4e30-b154-c3613ecf9d60/predictions'},
 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/339692d8-b809-4e30-b154-c3613ecf9d60/predictions'],
 'state': 'ready',
 'cp4d_url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/339692d8-b809-4e30-b154-c3613ecf9d60/predictions?version=2023-11-21'}

In [35]:
with open("../config/deployment_info.yml","w") as file:
    yaml.dump(deployment_endpoint, file)